<a href="https://colab.research.google.com/github/garciafido/cima-goes/blob/master/Mesoscale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q netCDF4
!pip install -q  pyproj
!pip install -q aioftp
!pip install -q aiofiles
# Cartopy
!apt-get -q install libproj-dev proj-data proj-bin
!apt-get -q install libgeos-dev
!pip install -q  cython
!pip install -q  cartopy

     |████████████████████████████████| 4.1MB 4.8MB/s 
     |████████████████████████████████| 307kB 57.9MB/s 
     |████████████████████████████████| 9.8MB 4.8MB/s 
Reading package lists...
Building dependency tree...
Reading state information...
proj-data is already the newest version (4.9.3-2).
proj-data set to manually installed.
The following NEW packages will be installed:
  libproj-dev proj-bin
0 upgraded, 2 newly installed, 0 to remove and 14 not upgraded.
Need to get 232 kB of archives.
After this operation, 1,220 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libproj-dev amd64 4.9.3-2 [199 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 proj-bin amd64 4.9.3-2 [32.3 kB]
Fetched 232 kB in 1s (208 kB/s)
Selecting previously unselected package libproj-dev:amd64.
(Reading database ... 131183 files and directories currently installed.)
Preparing to unpack .../libproj-dev_4.9.3-2_amd64.deb ...
Unpacking libproj

In [2]:
!pip uninstall -y -q cima.goes
!pip install -q git+https://github.com/garciafido/cima-goes.git
!pip show cima.goes

Name: cima.goes
Version: 1.1b62
Summary: GOES-16 File Processing
Home-page: https://github.com/garciafido/cima-goes
Author: Fido Garcia
Author-email: garciafido@gmail.com
License: MIT
Location: /usr/local/lib/python3.6/dist-packages
Requires: 
Required-by: 


In [9]:
import json
from dataclasses import asdict

import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from cima.goes.utils import timeit
from cima.goes.storage import GCS, NFS, FTP
from cima.goes import ProductBand, Product, Band
from cima.goes.projects import BatchProcess, DatesRange, HoursRange
from cima.goes.tiles import get_data, get_lats_lons, LatLonRegion
from cima.goes.img import get_image_stream, stream2pil, CLOUD_TOPS_PALETTE


def show(image_stream):
    image = stream2pil(image_stream)
    plt.axis('off')
    plt.imshow(image)
    plt.show()


def save(image_stream, filepath, storage):
    storage.upload_stream(image_stream, filepath)


def data_vis(data):
    albedo = (data * np.pi * 0.3) / 663.274497
    return np.power(albedo, 1.5)


def data_ir(data):
    return data - 273


def process_minute(gcs,
                   year, month, day, hour, minute,
                   band_blobs):
    title = f'{year}/{month:02d}/{day:02d} {hour:02d}:{minute:02d}'
    print('Begin:', title)

    #
    # Se usa ftp para guardar los archivos generados
    #
    storage, base_path = use_ftp()

    #
    # Se genera el png correspondiente a VIS
    #
    dataset = gcs.get_dataset(band_blobs[(Product.RadM, 2)])
    try:
        data = get_data(dataset)
        data = data_vis(data)
        lats, lons = get_lats_lons(dataset)
    finally:
        dataset.close()

    image_stream = get_image_stream(
        data, lats, lons,
        vmin=0,
        vmax=0.7,
        cmap='gray')
    filename = f'{year:04d}_{month:02d}_{day:02d}_{hour:02d}_{minute:02d}_VIS.png'
    filepath = f'{base_path}/{year:04d}/{month:02d}/{day:02d}/{hour:02d}/{filename}'
    save(image_stream, filepath, storage)
    region = LatLonRegion(
        lat_north=np.max(lats),
        lat_south=np.min(lats),
        lon_west=np.min(lons),
        lon_east=np.max(lons)
    )
    #
    # Guarda la región correspondiente a la imagen
    #
    jsonpath = filepath[:-8] + '_region.json'
    storage.upload_data(bytes(json.dumps(asdict(region), indent=2), 'utf8'), jsonpath)
    #     show(image_stream)

    #
    # Se genera el png correspondiente a IR
    #
    dataset = gcs.get_dataset(band_blobs[(Product.CMIPM, 13)])
    try:
        data = get_data(dataset)
        data = data_ir(data)
        lats, lons = get_lats_lons(dataset)
    finally:
        dataset.close()
    image_stream = get_image_stream(
        data, lats, lons,
        vmin=-90,
        vmax=50,
        cmap=CLOUD_TOPS_PALETTE)
    filename = f'{year:04d}_{month:02d}_{day:02d}_{hour:02d}_{minute:02d}_IR.png'
    filepath = f'{base_path}/{year:04d}/{month:02d}/{day:02d}/{hour:02d}/{filename}'
    save(image_stream, filepath, storage)
#     show(image_stream)


def get_credentials(ftp):
    CREDENTIALS_FILEPATH = '/OTs/gcs_credentials.json'
    data = ftp.download_data(CREDENTIALS_FILEPATH)
    return json.loads(data.decode('UTF-8'))


def use_ftp():
    FTP_HOST = '<Your Server URL>'
    FTP_USER = '<Your User Name>'
    FTP_PASSWORD = '<Your Password>'

    return FTP(
        host=FTP_HOST,
        user=FTP_USER,
        password=FTP_PASSWORD), '/OTs/mesoscale/'


def use_nfs():
    return NFS(), '/content/mesoscale/'


@timeit
def run_batch():
    ftp, base_path = use_ftp()
    gcs = GCS(credentials_as_dict=get_credentials(ftp))

    #
    # Se define un proceso batch para 2 Bandas (2 y 13) y para los rangos de
    # fecha, hora, minuto que se encuentren en range1 y range2.
    #

    range1 = DatesRange(
        date(2018, 11, 10),  # Desde el dia
        date(2018, 11, 10),  # Hasta el dia
        [HoursRange(15, 18)],  # Rangos de horas
        '15a18'  # Nombre del log
    )

    range2 = DatesRange(
        date(2018, 11, 10),  # Desde el dia
        date(2018, 11, 10),  # Hasta el dia
        [HoursRange(19, 22)],  # Rangos de horas
        '19a22'  # Nombre del log
    )

    MACHINE_NAME = 'COLAB_SAMPLE'
    batch = BatchProcess(
        gcs,
        [
            ProductBand(Product.CMIPM, 13, subproduct=2),
            ProductBand(Product.RadM, 2, subproduct=2),
        ],
        [range1, range2],  # Rangos de fechas y horas
        ftp,  # Log en ftp para continuar a partir del último día procesado
        '/OTs/logs/',  # Directorio de logs
        MACHINE_NAME,  # Nombre de PC que ejecuta este batch
    )

    #
    # Se ejecuta la función process_minute en 2 proecesos paralelos
    # Esta función será llamada por cada minuto dentro de range1 y range2
    #

    results = batch.run(process_minute, workers=2)
    print(results)


run_batch()


Task exception was never retrieved
future: <Task finished coro=<_run() done, defined at /usr/local/lib/python3.6/dist-packages/cima/goes/tasks/_concurrent.py:21> exception=KeyboardInterrupt()>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-a8a362d701a2>", line 170, in <module>
    run_batch()
  File "/usr/local/lib/python3.6/dist-packages/cima/goes/utils/_timeit.py", line 7, in timed
    result = method(*args, **kwargs)
  File "<ipython-input-4-a8a362d701a2>", line 166, in run_batch
    results = batch.run(process_minute, workers=2)
  File "/usr/local/lib/python3.6/dist-packages/cima/goes/projects/_project.py", line 201, in run
    return run_concurrent(tasks, workers)
  File "/usr/local/lib/python3.6/dist-packages/cima/goes/tasks/_concurrent.py", line 42, in run_concurrent
    results = loop.run_until_complete(futur

KeyboardInterrupt: ignored